In [3]:
import numpy as np
from mmda.utils.cca_class import NormalizedCCA
from omegaconf import OmegaConf
from pathlib import Path
# ruff: noqa: ERA001

In [9]:
# As cca_zoo does not have the normalized (zero-meaned) the input embeddigns of CCA, we re-wrote a NormalizedCCA class
cca = NormalizedCCA() # initialize the CCA class

# Due to the rank constraint of CCA, the number of training samples should be larger
# than the number of dimensions and sim_dim (see line 15).
# The number of samples should be the same for both modalities
train_emb_modal1 = np.random.rand(1500, 1024) # 1500 samples, 1024 dimensions
train_emb_modal2 = np.random.rand(1500, 768) # 1500 samples, 768 dimensions

test_emb_modal1 = np.random.rand(500, 1024) # 500 samples, 1024 dimensions
test_emb_modal2 = np.random.rand(500, 768) # 500 samples, 768 dimensions

cfg = OmegaConf.load("../config/main.yaml")
cfg_dataset = cfg["cosmos"] # we use the COSMOS dataset as an example
# cfg_dataset should have (from config/main.yaml):
#   sim_dim: 700 # dimension of the similarity score and the CCA transformation
#   equal_weights: False # whether to use equal weights for the similarity score or weighted cosine similarity
#   img_encoder: "dino" # the image encoder to load the image embeddings, not used in the CCA module
#   text_encoder: "gtr" # the text encoder to load the text embeddings, not used in the CCA module
#   paths:
#     dataset_path: "/nas/pohan/datasets/COSMOS/"
#     save_path: ${cosmos.paths.dataset_path}embeddings/
#     plots_path: ${repo_root}plots/COSMOS/


cca.fit_transform_train_data(cfg_dataset, train_emb_modal1, train_emb_modal2)
print(f"Fit the CCA model. CCA dimension: {cfg_dataset.sim_dim}")
print(f"Train data shape: {train_emb_modal1.shape}, {train_emb_modal2.shape}")

# transform the test data
test_emb_modal1, test_emb_modal2 = cca.transform_data(test_emb_modal1, test_emb_modal2)
print(f"Test data shape: {test_emb_modal1.shape}, {test_emb_modal2.shape}")

# save the CCA model
cca_save_path = Path(cfg_dataset.paths.save_path) / "test_cca_model.pkl"
cca.save_model(cca_save_path)  # save the class object for later use
print(f"Save the CCA model to {cca_save_path}.")

# load the CCA model
# cca.load_model(cca_save_path)
# print(f"Load the CCA model from {cca_save_path}.")


Fit the CCA model. CCA dimension: 700
Train data shape: (1500, 1024), (1500, 768)
Test data shape: (500, 700), (500, 700)
Save the CCA model to /nas/pohan/datasets/COSMOS/embeddings/test_cca_model.pkl.
